In [1]:
import pandas as pd
import math
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
influence_pd=pd.read_csv('influence_data.csv')
artist_pd=pd.read_csv('data_by_artist.csv')
artist_pd=artist_pd.drop(['mode','key','duration_ms','popularity','count'],axis=1)
artist_pd

,artist_name,artist_id,danceability,energy,valence,tempo,loudness,acousticness,instrumentalness,liveness,speechiness
0,Frank Sinatra,792507,0.384478,0.238017,0.364288,110.181698,-14.271141,0.735648,0.020855,0.232106,0.049614
1,Vladimir Horowitz,119107,0.343210,0.118844,0.225951,94.900679,-23.193418,0.990070,0.879508,0.183812,0.043360
2,Johnny Cash,816890,0.619803,0.449381,0.680662,115.037747,-11.593104,0.685637,0.022647,0.242243,0.098216
3,Billie Holiday,79016,0.572637,0.201368,0.498934,109.912172,-13.225966,0.908499,0.013064,0.217727,0.062432
4,Bob Dylan,66915,0.512598,0.477932,0.551934,126.160149,-11.184330,0.562567,0.034211,0.308978,0.064535
...,...,...,...,...,...,...,...,...,...,...,...
5849,Natalie La Rose,3359519,0.830000,0.520000,0.735000,104.990000,-8.714000,0.000792,0.000013,0.065600,0.037600
5850,Sarah Ross,3381566,0.721000,0.944000,0.626000,85.002000,-5.982000,0.013000,0.000000,0.320000,0.159000
5851,Rotimi,3410250,0.637000,0.501000,0.431000,103.993000,-6.148000,0.229000,0.000059,0.099000,0.187000
5852,Jillian Jacqueline,3455945,0.547000,0.672000,0.283000,155.791000,-5.023000,0.304000,0.000000,0.099600,0.049600


In [3]:
for col in artist_pd.columns:
    if(col=='artist_name' or col=='artist_id'): continue
    artist_pd[col]=(artist_pd[col]-artist_pd[col].mean())/(artist_pd[col].std())

In [4]:
artist_attr={}
for i in range(artist_pd.shape[0]):
    artist_attr[artist_pd.loc[i].artist_id]=artist_pd.loc[i].drop('artist_id').to_dict()
restore={}

In [6]:
def calc1(u,v):
    if(restore.setdefault(u)==None): restore[u]={}
    if(restore.setdefault(v)==None): restore[v]={}
    if(restore[u].setdefault(v)==None):
        atu=artist_attr[u]
        atv=artist_attr[v]
        n,sum=0,0
        for key in atu.keys():
            if(key=='artist_name'): continue
            n+=1
            sum+=math.pow((atu[key]-atv[key]),2)
        if(sum==0): return 0
        w=1/math.sqrt(sum/n)
        restore[u][v]=w
        restore[v][u]=w
        return w
    else:
        return restore[u][v]
def calc2(u,v):
    cnt,w=0,0
    for i in range(artist_pd.shape[0]):
        vid=artist_pd.loc[i].artist_id
        if(vid==u or vid==v): continue
        tmp=calc1(u,vid)
        if(tmp!=0):
            cnt+=1
            w+=tmp
    return w/cnt

score=[0,0]
for i in range(influence_pd.shape[0]):
    if(i%100==0): print(i)
    u=influence_pd.loc[i].influencer_id
    v=influence_pd.loc[i].follower_id
    if(artist_attr.setdefault(v)==None): continue
    score[0]+=calc1(u,v)
    score[1]+=calc2(u,v)

0


KeyboardInterrupt: 

In [12]:
artist_pd=artist_pd.drop(['artist_name'],axis=1)

In [25]:
with open('artist_pd.txt',"w+",encoding='utf-8') as f:
    f.write(str(artist_pd.shape[0])+"\n")
    for i in range(len(artist_pd)):
        for col in artist_pd.loc[i].to_list():
            f.write(str(col)+" ")
        f.write("\n")

In [19]:
artist_pd[['artist_id']]=artist_pd[['artist_id']].astype(int)